In [ ]:
import sys
sys.executable

In [ ]:
import os
import json
import csv
import pymongo
import musicbrainzngs
import arrow
import re
from datetime import datetime
from pymongo import MongoClient
from collections import defaultdict, OrderedDict

from dateutil import parser
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
DATETIME_FORMAT = '%Y-%m-%dT%H:%M:%S'
# setup mongo
client = MongoClient()
db = client["bandstour"]
venues =  db["selectedVenues"]
artists =  db["selectedArtists"]

def query_yes_no(question, default="yes"):
    """Ask a yes/no question via raw_input() and return their answer.
    "question" is a string that is presented to the user.
    "default" is the presumed answer if the user just hits <Enter>.
        It must be "yes" (the default), "no" or None (meaning
        an answer is required of the user).
    The "answer" return value is one of "yes" or "no".
    """
    valid = {"yes":True,   "y":True,  "ye":True,
             "no":False,     "n":False}
    if default == None:
        prompt = " [y/n] "
    elif default == "yes":
        prompt = " [Y/n] "
    elif default == "no":
        prompt = " [y/N] "
    else:
        raise ValueError("invalid default answer: '%s'" % default)

    while True:
        sys.stdout.write(question + prompt)
        choice = raw_input().lower()
        if default is not None and choice == '':
            return valid[default]
        elif choice in valid:
            return valid[choice]
        else:
            sys.stdout.write("Please respond with 'yes' or 'no' "\
"(or 'y' or 'n').\n")

def slugify(s):
    """
    Simplifies ugly strings into something URL-friendly.
    >>> print slugify("[Some] _ Article's Title--")
    some-articles-title
    """

    s = re.sub('\s+', ' ', s)
    s = s.replace(' ', '-')

    s = re.sub('\W+', ' ', s)

    s = s.strip()

    s = s.lstrip()
    s = re.sub('\W+', '_', s)
    return s





In [ ]:
print "RUNNING DEDUP SMART .018"
Amount=0
venueB=0
foundVenueDUP=False
for venueA in venues.find({},no_cursor_timeout=True).sort("longitude",1) :
        #print venueA["id"]
        #if venueB!=0:
            #print "DUP name to test", "venueB",venueB["_id"]
        if venueB!=0 and round(venueA["latitude"],3) == round(venueB["latitude"],3) and round(venueA["longitude"],3)== round(venueB["longitude"],3) and (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 90 and venueA["count"] <= venueB["count"]) :
            print venueA["count"],"\"",venueA["_id"],"\"","/","\"", venueB["_id"],"\"",venueB["count"],
            foundVenueDUP=True
            for artist in artists.find():
                            counterREMPLC_ART=0
                            for gig   in artist["gigs"]:
                                if (gig["venue"]["name"]==venueA["name"] and round(gig["venue"]["latitude"],1)== round(venueA["latitude"],1) and round(gig["venue"]["longitude"],1)== round(venueA["longitude"],1) ):
                                    #print artist["_id"] , gig["venue"]["name"],"****************************************************"
                                    gigTMPdedup=gig

                                    artist["gigs"].remove(gig)
                                    gigTMPdedup["venue"]=venueB
                                    counterREMPLC_ART+=1
                                    artist["gigs"].append(gigTMPdedup)

                            
                            if  counterREMPLC_ART>0 :
                                #print ",",counterREMPLC_ART,",",
                                artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
                                
            venueB["count"] += venueA["count"]  
            venues.update({"_id" :venueB["_id"]}, venueB, upsert=True, multi=False)
                         #       if venueB["count"] == 0:
            venues.remove({"_id" :venueA["_id"]})
            print "REMOVED"
            Amount+=1
        
        
        
        if foundVenueDUP==False:
            venueB=venueA
        else:
            foundVenueDUP=False
            #print "DUP name stays", "venueB"," \"",venueB["_id"],"\""
   
        
            
           
print "THE END"
print Amount


In [ ]:
print venues.findOne({})

In [ ]:
print "RUNNING DEDUP SMART .25"

def dedupVenue(param1,param2):
 Amount=0
 venueB=0
 foundVenueDUP=False
 for venueA in venues.find({},no_cursor_timeout=True).sort(param1,param2) :
        #print venueA["id"]
        #if venueB!=0:
            #print "DUP name to test", "venueB",venueB["_id"]
        if venueB!=0 and round(venueA["latitude"],3) == round(venueB["latitude"],3) and round(venueA["longitude"],3)== round(venueB["longitude"],3) and (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 90 and venueA["count"] <= venueB["count"]) :
            print venueA["count"],"\"",venueA["_id"],"\"","/","\"", venueB["_id"],"\"",venueB["count"],
            foundVenueDUP=True
            for artist in artists.find():
                            counterREMPLC_ART=0
                            for gig   in artist["gigs"]:
                                if (gig["venue"]["name"]==venueA["name"] and round(gig["venue"]["latitude"],1)== round(venueA["latitude"],1) and round(gig["venue"]["longitude"],1)== round(venueA["longitude"],1) ):
                                    #print artist["_id"] , gig["venue"]["name"],"****************************************************"
                                    gigTMPdedup=gig

                                    artist["gigs"].remove(gig)
                                    gigTMPdedup["venue"]=venueB
                                    counterREMPLC_ART+=1
                                    artist["gigs"].append(gigTMPdedup)

                            
                            if  counterREMPLC_ART>0 :
                                #print ",",counterREMPLC_ART,",",
                                artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
                                
            venueB["count"] += venueA["count"]  
            venues.update({"_id" :venueB["_id"]}, venueB, upsert=True, multi=False)
                         #       if venueB["count"] == 0:
            venues.remove({"_id" :venueA["_id"]})
            print "REMOVED"
            Amount+=1
        
        
        
        if foundVenueDUP==False:
            venueB=venueA
        else:
            foundVenueDUP=False
            #print "DUP name stays", "venueB"," \"",venueB["_id"],"\""
   
        
            
           
 print "THE END"
 return Amount

In [ ]:
import random
while True:  # making a loop

            DEDUPME=0
            parmod=["latitude","longitude","_id","city"]
            parsens=[1,-1]
            parm1=random.choice(parmod)
            parm2=random.choice(parsens)
            print parm1,parm2
            result=dedupVenue(parm1,parm2)
            print result



##CELLULES SUIVANTE POUR MEMOIRE ET COMMENTAIRE

In [ ]:
#WE WANT TO LOOP OVER VENUES TO ELIMINATE RESIDUAL TEST DATAS (BINTOWN) AND  FIX THIS, AND DEDUPLICATE VENUES
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
for venueA in venues.find({},no_cursor_timeout=True) :
    
    #FIRST THE ONES SURELY WRONG:
    
    
    if  (venueA["name"] in ["TEST NEW VENUE","TEST VENUE","Test ","Test 8","Test Facility","Test German Venue","Test Only","Test Vanue","Test Venu","Test Venue","Test Venue 1","Test Venue DJ 11","Test Venue Marquee","Test de Marco","Test event","Test venue","Test venue 62","Test_Venue_100708","Testing a Venue","test again vm","test event","test ire venue","test test test","test test test 1","test test test 2","test venue ","test venue 9","test2","test3","testdate","teste"]  ):
        ####extracted via salles_a_trier.ods
        ###and (venueA["latitude"]!= "37.7909889" and venueA["longitude"]!= "-122.3990707"))
            print "AAAA  =>", venueA["_id"]
            print slugify(venueA["_id"])
            print venueA
            #check if this test venue exists and it is a typo
            print " TO DELETE, NOW SHOWING ARTISTS TO ALTER"

            for artist in artists.find():
                             
                                 for gig   in artist["gigs"]:
                                    if (gig["venue"]["name"]==venueA["_id"] and round(gig["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gig["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                        #[{u'venue': {u'city': u'Montreal', u'name': u'Medley', u'country': u'Canada', u'region': u'QC', u'longitude': -73.554392, u'place': u'Medley', u'latitude': 45.512288}, u'datetime': datetime.datetime(2002, 4, 14, 20, 30, 16)}, {u'venue': {u'city': u'Boston', u'name': u'Test Venue', u'country': u'United States', u'region': u'MA', u'longitude': -71.0602778, u'place': u'Test Venue', u'latitude': 42.3583333}, u'datetime': datetime.datetime(2010, 1, 2, 19, 0)}]
                                        
                                        
                                        print len(artist["gigs"])
                                        
                                        print "_______________"
                                        print gig
                                        print "_-_-_-_-_-_-_-_-"
                                        print artist["_id"] , gig["venue"],"****************************************************"    
                                        print "______---------------__________"
                                        print "REMOVING"
                                        artist["gigs"].remove(gig)
                                        print len(artist["gigs"])
                                        artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
            print "DONE WITH THIS VENUE1"           
            
            

        

In [ ]:
     #THEN THE PROBABLY RIGHT ONES:       
            
for venueA in venues.find({},no_cursor_timeout=True) :
    
    #we check for test venues
    if  (venueA["name"] in ["Test","Test (new)","Test 17:01","Test Again","Test Club","Test Past Event","Test Vannue","Test Venues","Test Venz","Test Xavier","Test2","Test3","TestVenue","test ven","test venue 1","testing"] ) :
        ####extracted via salles_a_trier.ods
            print "AAAA  =>", venueA["_id"]
            print slugify(venueA["_id"])
            print venueA
            #check if this test venue exists and it is a typo
            counter=0
            for artist in artists.find():
                            for gig   in artist["gigs"]:
                                if ( round(gig["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gig["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                    counter+=1
                                    #print artist["_id"] , gig["venue"],"****************************************************"
                                    #print counter
                                    if counter==20:
                                        break
                            if counter==20:
                                print venueA["name"]
                                print venueA,"could be replaced       with      ",gig["venue"]
                                print gig["venue"]["name"]
                                print "NOW SHOWING ARTISTS TO ALTER"
                                venueTMP=gig["venue"]
                                break
            for artist in artists.find():
                                 
                                 modded=0
                                 for gigB   in artist["gigs"]:
                                    if (gigB["venue"]["name"]==venueA["name"] and round(gigB["venue"]["latitude"],6)== round(venueA["latitude"],6) and round(gigB["venue"]["longitude"],6)== round(venueA["longitude"],6) ):
                                        print "GIG",gigB
                                        modded=1
                                        gigTMP=gigB
                                        artist["gigs"].remove(gigB)
                                        if "venueTMP" in locals():
                                            gigTMP["venue"]=venueTMP
                                            print "GIGTMP",gigTMP
                                            artist["gigs"].append(gigTMP)
                                        #print artist["_id"] , artist["gigs"],"****************************************************"    
                                        
                                    else:
                                        pass
                                 if modded==1:
                                    pass
                                    print "MODDED"
                                    #print artist
                                    artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False) 
            if "venueTMP" in locals():
                del  venueTMP                       
            print "DONE WITH THIS VENUE2"           
            

In [ ]:

for venueA in venues.find({},no_cursor_timeout=True) :
    if  "count" not in venueA:
        
                print venueA, "NO COUNT!!!!!!!!!"
    elif  isinstance(venueA["count"], (int, long)):
        

In [ ]:
###....AND DEDUPLICATE VENUES
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
print "DEDUPLICATOR V.70"
myFile = open('MY_LIST_OF_DUPS.csv', 'w')

MY_LIST_OF_DUPS=[]
for venueA in venues.find({},no_cursor_timeout=True) :
            
    
    #we check for test venues
    
            for venueB in venues.find({},no_cursor_timeout=True) :
                if venueA["_id"]==venueB["_id"]:
                    pass
                    #print venueA["_id"],'SAME'
                elif (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 75 and venueA["count"] >= venueB["count"]) :
                    
           
                    
                    if ( round(venueA["latitude"],1)!= round(venueB["latitude"],1) and round(venueA["longitude"],1)!= round(venueB["longitude"],1) ):
                        #print "TOO FAR, SKIPPED"
                        pass
                    else:
                        print "_______________________________"
                        print "AAAA  =>", venueA["_id"]
                        #print venueA
                        #print "AAAA slug  =>",slugify(venueA["_id"]).lower()
                        #print "AAAA lat/long =>",venueA["latitude"],venueA["longitude"]
                        print "appxAAA ltlng=>",round(venueA["latitude"],3),round(venueA["longitude"],3)
                        print "AAAA gigs =>", venueA["count"]
                        print "BBBB  =>",venueB["_id"]
                        #print venueB
                        #print "BBBB slug  =>",slugify(venueB["_id"]).lower()
                        #print "BBBB lat/long =>",venueB["latitude"],venueB["longitude"]
                        print "appxBBB ltlng=>",round(venueB["latitude"],3),round(venueB["longitude"],3)
                        print "BBBB gigs =>", venueB["count"]
                        MY_LIST_OF_DUPS.append([venueA["_id"], venueB["_id"]])
                        
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(MY_LIST_OF_DUPS)
     
print("Writing DUPS list complete")                        
                        
     

In [ ]:
print "DEDUPLICATOR V.70"
myFile = open('MY_LIST_OF_DUPS.csv', 'w')

MY_LIST_OF_DUPS=[]
for venueA in venues.find({},no_cursor_timeout=True).sort("_id") :
            print "\"", venueA["id"],"\""

In [ ]:
###....AND DEDUPLICATE VENUES
#ex: "   dede" =add his dates to> "dede" who already has his.
#in case there is an ambiguity, we will have to choose
print "DEDUPLICATOR V.70"
myFile = open('MY_LIST_OF_DUPS.csv', 'w')

MY_LIST_OF_DUPS=[]
for venueA in venues.find({},no_cursor_timeout=True).sort("_id") :
            
    
    #we check for test venues
    
            for venueB in venues.find({},no_cursor_timeout=True).sort("_id") :
                if venueA["_id"]==venueB["_id"]:
                    pass
                    #print venueA["_id"],'SAME'
                if 
                
                
                elif (fuzz.ratio(\
                                        slugify(venueA["_id"]).lower()\
                                        , slugify(venueB["_id"]).lower()\
                                       ) > 75 and venueA["count"] >= venueB["count"]) :
                    
           
                    
                    if ( round(venueA["latitude"],1)!= round(venueB["latitude"],1) and round(venueA["longitude"],1)!= round(venueB["longitude"],1) ):
                        #print "TOO FAR, SKIPPED"
                        pass
                    else:
                        print "_______________________________"
                        print "AAAA  =>", venueA["_id"]
                        #print venueA
                        #print "AAAA slug  =>",slugify(venueA["_id"]).lower()
                        #print "AAAA lat/long =>",venueA["latitude"],venueA["longitude"]
                        print "appxAAA ltlng=>",round(venueA["latitude"],3),round(venueA["longitude"],3)
                        print "AAAA gigs =>", venueA["count"]
                        print "BBBB  =>",venueB["_id"]
                        #print venueB
                        #print "BBBB slug  =>",slugify(venueB["_id"]).lower()
                        #print "BBBB lat/long =>",venueB["latitude"],venueB["longitude"]
                        print "appxBBB ltlng=>",round(venueB["latitude"],3),round(venueB["longitude"],3)
                        print "BBBB gigs =>", venueB["count"]
                        MY_LIST_OF_DUPS.append([venueA["_id"], venueB["_id"]])
                        
with myFile:
    writer = csv.writer(myFile)
    writer.writerows(MY_LIST_OF_DUPS)
     
print("Writing DUPS list complete")                        
                        
     

In [ ]:
                   
                        
                        
                        Answer_art= query_yes_no("MERGE "+venueB["_id"]+ " WITH "+ venueA["_id"]+" ?" , default="no")
                        print Answer_art
                        if Answer_art == True :
                         
                         for artist in artists.find():
                            counterREMPLC_ART=0
                            for gig   in artist["gigs"]:
                                if (gig["venue"]["name"]==venueB["name"] and round(gig["venue"]["latitude"],1)== round(venueB["latitude"],1) and round(gig["venue"]["longitude"],1)== round(venueB["longitude"],1) ):
                                    print artist["_id"] , gig["venue"],"****************************************************"
                                    gigTMPdedup=gig
                                    counterREMPLC_ART+=1
                                    #print "*-------"
                                    #print gig
                                    #print "----------"
                                    artist["gigs"].remove(gig)
                                    gigTMPdedup["venue"]=venueA
                                    
                                    #gigTMPdedup["venue"].pop("count",None )
                                    #gigTMPdedup["venue"].pop("id",None )
                                    artist["gigs"].append(gigTMPdedup)
                                    #print gigTMPdedup
                                    #print venueA
                                    #print "venuecountCHECK",venueA["count"],venueB["count"]
                                    #venueA["count"] += 1
                                    #venueB["count"] -= 1
                                    #print "venuecountCHECK",venueA["count"],venueB["count"]
                                    #[4 if x==1 else x for x in a]
                            
                            if counterREMPLC_ART > 0 :
                                print counterREMPLC_ART
                                artists.update({"_id" :artist["_id"]}, artist, upsert=True, multi=False)
                                
                         venueA["count"] += venueB["count"]  
                         venues.update({"_id" :venueA["_id"]}, venueA, upsert=True, multi=False)
                         #       if venueB["count"] == 0:
                         venues.remove({"_id" :venueB["_id"]})
                         #           print venueB["name"] ,"removed"
                         #       else:
                         #           print venueB["name"] ,"updated"
                         #                                           
                         #           venues.update({"_id" :venueB["_id"]}, venueB, upsert=True, multi=False)
                            
                    

In [ ]:
##CHECK FOR DATE DUPLICATES
for artist in artists.find():
    seen = set()
    uniq = []
    counter=0
    for gig in artist["gigs"]:
         counter +=1
         if gig["datetime"] not in seen:
            uniq.append(x)
            seen.add(x)
    if   counter != len(uniq) :
        print "DUP"http://localhost:8888/notebooks/3_cleaner/2_venue_deduplicator_WIP-Copy1.ipynb#
        break
        
        
    